In [6]:
from collections import defaultdict
from datetime import datetime

In [7]:

# Copy mods.json to mods-YYYY-MM-dd-HH-MM-SS.json
import json
import shutil
shutil.copyfile('mods.json', f'mods-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.json')

# Load configuration and mods files.
with open('config/1843965353/config.json', 'r') as cf:
    config_file = json.load(cf)
mod_to_author = config_file['ModIdToAuthorId']

with open('mods.json', 'r') as mf:
    mods_file = json.load(mf)
mods = mods_file['mods']

# For any mod that doesn't have an author mapping, create an author 'Author of <mod name>'
for mod in mods:
    if mod['label']['id'] not in mod_to_author:
        mod_to_author[mod['label']['id']] = f"Author of {mod['label']['id']}"


In [8]:
# Show the number of unique authors and mods
print(f"Authors: {len(set(mod_to_author.values()))}")
print(f"Mods: {len(mods)}")

Authors: 47
Mods: 127


In [9]:
# Group mods by author.
grouped = {author: [mod for mod in mods if mod_to_author[mod['label']['id']] == author] 
           for author in set(mod_to_author.values())}

# Sort authors by the number of enabled mods they have.
# Enabledness of a mod can be determined if the mod['enabledForDlc'] array contains any strings.
grouped = dict(sorted(grouped.items(), key=lambda item: sum(100 for mod in item[1] if mod['enabledForDlc'] != []) + len(item[1]), reverse=True))

# If Mod Manager is enabled (mod ID 1843965353), put it at the top of the list, along with its author (76561198082968733).
if '76561198082968733' in grouped and any(mod['label']['id'] == '1843965353' and mod['enabledForDlc'] != [] for mod in mods):
    ony_mods = grouped.pop('76561198082968733')
    ony_mods = [mod for mod in ony_mods if mod['label']['id'] == '1843965353'] + [mod for mod in ony_mods if mod['label']['id'] != '1843965353']
    grouped = {'76561198082968733': ony_mods, **grouped}

# If Mod Updater is enabled (mod ID 2018291283), put it at the top of the list, along with its author (76561198025154321).
if '76561198025154321' in grouped and any(mod['label']['id'] == '2018291283' and mod['enabledForDlc'] != [] for mod in mods):
    mu_mods = grouped.pop('76561198025154321')
    mu_mods = [mod for mod in mu_mods if mod['label']['id'] == '2018291283'] + [mod for mod in mu_mods if mod['label']['id'] != '2018291283']
    grouped = {'76561198025154321': mu_mods, **grouped}

# Print the number of mods per author, along with how many of their mods are enabled.
for author, author_mods in grouped.items():
    enabled_count = sum(1 for mod in author_mods if mod['enabledForDlc'] != [])
    author_name = config_file['SteamIdToAuthorInfo'].get(author, {}).get('Name', author)
    print(f"{author_name}: {len(author_mods)} mods, {enabled_count} enabled")

Stephen: 22 mods, 19 enabled
Sanchozz: 13 mods, 12 enabled
Aze: 9 mods, 9 enabled
Sgt_Imalas: 7 mods, 7 enabled
Cairath: 5 mods, 5 enabled
llunak: 5 mods, 5 enabled
Glampi: 4 mods, 4 enabled
Ony: 5 mods, 3 enabled
Aki: 3 mods, 3 enabled
Identifier: 3 mods, 3 enabled
asquared31415: 5 mods, 2 enabled
shelena_malk: 4 mods, 2 enabled
Swistak: 3 mods, 2 enabled
Fumihiko: 3 mods, 2 enabled
mesi: 2 mods, 2 enabled
Skyrunner: 3 mods, 1 enabled
YMS: 2 mods, 1 enabled
全宇宙的牙医: 1 mods, 1 enabled
Heinermann: 1 mods, 1 enabled
Dr. Chapman: 1 mods, 1 enabled
NatoTomato: 1 mods, 1 enabled
avil: 1 mods, 1 enabled
Pholith: 1 mods, 1 enabled
3GuB: 1 mods, 1 enabled
gbdickinson: 1 mods, 1 enabled
Malc: 1 mods, 1 enabled
Pope Snowball: 1 mods, 1 enabled
NNL: 1 mods, 1 enabled
Arya: 1 mods, 1 enabled
SnackJunkie: 1 mods, 1 enabled
Matze149: 1 mods, 1 enabled
Romen: 4 mods, 0 enabled
pether.pg: 3 mods, 0 enabled
Davkas: 2 mods, 0 enabled
counter3d: 2 mods, 0 enabled
Hex: 1 mods, 0 enabled
scitor: 1 mods, 0 e

In [10]:
# Make a deep copy of mods_file.
mods_file = mods_file.copy()

# Replace the 'mods' list with the grouped mods.
mods_file['mods'] = [mod for mods in grouped.values() for mod in mods]

# Write the reordered mods back to mods.json.
with open('mods.json', 'w') as out_file:
    json.dump(mods_file, out_file, indent=2, ensure_ascii=False)